In [2]:
from IPython.core.display import display
import pyodbc
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules, fpmax, fpgrowth
from Query import Query

from configuration import Configuration
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 0)

conn = pyodbc.connect(Configuration.getConfigValue('database_connection'))
supportStr = Configuration.getConfigValue('QUESTION3_METRIC1_SUPORT')
support = float(supportStr)
query = Query.getQuery('query_3_1')
result = pd.read_sql_query(query,conn)
dataframes = pd.DataFrame(result)

if(len(dataframes) == 0):
    display("No data to analyze")
else:
    dataframes = pd.pivot_table(dataframes,  index="ID_COMMIT", columns=['RULE_DESCRIPTION'],values="VALOR")
    dataset = []

    for i in range(0, len(dataframes)):
        datasetI = []
        for j in range(0, len(dataframes.columns)):
            if str(dataframes.values[i,j]) == "1.0"  :
                datasetI.append( str(dataframes.columns[j]))
        dataset.append(datasetI)
        datasetI = []

    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df, min_support=0.05, use_colnames=True)
    display(frequent_itemsets)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.05)
    display(rules)


,support,itemsets
0,0.560645,(S1192 - String literals should not be duplicated)
1,0.454623,(S121 - Control structures should use curly braces)
2,0.064546,(S1142 - Functions should not contain too many return statements)
3,0.103732,(S3776 - Cognitive Complexity of functions should not be too high)
4,0.070568,(S1481 - Unused local variables should be removed)
5,0.050975,(S101 - Class names should comply with a naming convention)
6,0.185666,"(S121 - Control structures should use curly braces, S1192 - String literals should not be duplicated)"
7,0.066412,"(S1192 - String literals should not be duplicated, S3776 - Cognitive Complexity of functions should not be too high)"
8,0.061154,"(S121 - Control structures should use curly braces, S3776 - Cognitive Complexity of functions should not be too high)"
9,0.051315,"(S1192 - String literals should not be duplicated, S1481 - Unused local variables should be removed)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(S121 - Control structures should use curly braces),(S1192 - String literals should not be duplicated),0.454623,0.560645,0.185666,0.408396,0.728439,-0.069216,0.742651
1,(S1192 - String literals should not be duplicated),(S121 - Control structures should use curly braces),0.560645,0.454623,0.185666,0.331165,0.728439,-0.069216,0.815414
2,(S1192 - String literals should not be duplicated),(S3776 - Cognitive Complexity of functions should not be too high),0.560645,0.103732,0.066412,0.118457,1.141951,0.008255,1.016704
3,(S3776 - Cognitive Complexity of functions should not be too high),(S1192 - String literals should not be duplicated),0.103732,0.560645,0.066412,0.640229,1.141951,0.008255,1.221208
4,(S121 - Control structures should use curly braces),(S3776 - Cognitive Complexity of functions should not be too high),0.454623,0.103732,0.061154,0.134515,1.296755,0.013995,1.035567
5,(S3776 - Cognitive Complexity of functions should not be too high),(S121 - Control structures should use curly braces),0.103732,0.454623,0.061154,0.589534,1.296755,0.013995,1.328679
6,(S1192 - String literals should not be duplicated),(S1481 - Unused local variables should be removed),0.560645,0.070568,0.051315,0.091528,1.297013,0.011751,1.023071
7,(S1481 - Unused local variables should be removed),(S1192 - String literals should not be duplicated),0.070568,0.560645,0.051315,0.727163,1.297013,0.011751,1.610325
